In [4]:
!pip install scikit-learn requests tzdata

In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import re
import json
import pickle
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB 
from datetime import datetime
from typing import List

In [3]:
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi","Hello","Hey","Good morning","Good evening",
                         "What's up?","How are you?","Greetings","Yo","Hey there","How are you doing today?"],
            "responses": ["Hello! 👋","Hey there!","Hi, nice to meet you 😊",
                          "Greetings!","What's up?","How's your day going?","I'm doing great, how about you?"]
        },
        {
            "tag": "name",
            "patterns": ["What's your name?","What moniker what given to you by your creator?","Tell me your name",
                         "Do you have a name?","May I know your name?",
                         "What should I call you?","Are you a bot?",
                         "Introduce yourself","Name please","Tell me who you are"],
            "responses": ["I'm your friendly Moyennn!","They call me Moyennn!",
                          "I'm an AI assistant called Moyennn","Just a humble bot here to help 😊"]
        },
        {
            "tag": "time",
            "patterns": ["What time is it?","Tell me the time","Current time?",
                         "Can you give me the time?","Clock please",
                         "What's the hour?","Time right now?",
                         "Tell me the hour","Give me the time","Local time please"],
            "responses": []
        },
        {
            "tag": "weather",
            "patterns": ["What's the weather?","Weather today?","Is it hot outside?",
                         "Tell me the forecast","Do I need an umbrella?",
                         "What's the temperature?","How's the sky?",
                         "Weather in Lagos","Forecast please","Sunny today?","Weather in Akure"],
            "responses": []
        }
    ]
}


In [4]:
X = []
y = []

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        X.append(pattern)
        y.append(intent["tag"])

# Vectorize
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X)

# Training model
model = MultinomialNB()
model.fit(X_train, y)

print("Model trained successfully✅")


Model trained successfully✅


In [5]:
API_KEY = "8a611495b6b56f082f045d2ffed3389c"

def get_weather(user_input, default_city="Lagos"):
    # extract city from user input
    match = re.search(r'weather in (\w+)', user_input.lower())
    city = match.group(1).capitalize() if match else default_city
    
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city},NG&appid={API_KEY}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        temp = data["main"]["temp"]
        desc = data["weather"][0]["description"]
        return f"The weather in {city} is {desc} with {temp}°C 🌤️"
    else:
        return f"Sorry, I couldn’t fetch the weather for {city} 🌧️"

In [26]:
# Track the last conversation intent
last_intent = None  

def chatbot_response(user_input):
    global last_intent  
    responses = []
    answered_intents = set()

    # Split into smaller queries using punctuation and "and"
    parts = re.split(r'\s*(?:\?|\.|!|,|\band\b)\s*', user_input, flags=re.IGNORECASE)
    parts = [p.strip() for p in parts if p.strip()]

    context_triggered = False

    for part in parts:
        #logic to ignore all other context if one response has been handled
        if context_triggered:
            continue

        X_test = vectorizer.transform([part])
        tag = model.predict(X_test)[0]

        #Contextual Conversation Logic
        if last_intent == "greeting" and ("fine" in part.lower() or "good" in part.lower()):
            responses.append("I’m fine too! 😊 So, what can I do for you today?")
            last_intent = "smalltalk"
            context_triggered = True
            continue
       #to remove duplicate responses for the same intent
        if tag in answered_intents:
                continue

        #Normal Intent Handling
        if tag == "time":
            now = datetime.now()
            responses.append(f"The current time is {now.strftime('%H:%M:%S')} ⏰")
        
        elif tag == "weather":
            responses.append(get_weather(part))
        
        else:
            for intent in intents["intents"]:
                if intent["tag"] == tag:
                    if intent["responses"]:
                        # Greeting special case
                        if tag == "greeting":
                            responses.append("Hello! How are you doing today? ")
                        else:
                            responses.append(random.choice(intent["responses"]))
                        last_intent = tag
                        answered_intents.add(tag) # mark as answered
                    else:
                        responses.append("Hmm...")

    return " ".join(responses)


In [32]:
import pickle


with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [27]:
print(chatbot_response("the weather in Akure at the momment and what time is it?"))

The weather in Akure is overcast clouds with 25.56°C 🌤️ The current time is 12:19:35 ⏰


In [28]:
print(chatbot_response("Hi, how are you?"))

Hello! How are you doing today? 


In [29]:
print(chatbot_response("I'm fine thank you and you?"))

I’m fine too! 😊 So, what can I do for you today?
